![Logo](assets/logo.png)

Made by **Domonkos Nagy**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Fortuz/rl_education/blob/main/5.%20Temporal%20Difference/frozen_lake.ipynb)

# Frozen Lake

Frozen lake involves crossing a frozen lake from start to goal without falling into any holes by walking over the frozen lake. The player may not always move in the intended direction due to the slippery nature of the frozen lake.

The game starts with the player at location [0,0] of the frozen lake grid world with the goal located at far extent of the world e.g. [3,3] for the 4x4 environment.
Holes in the ice are distributed in set locations.
The player makes moves until they reach the goal or fall in a hole.

![Example image](assets/frozen_lake.png)

This problem can be formulated with a finite, undiscounted MDP, where the states are the positions in the grid world, the actions are UP, DOWN, LEFT and RIGHT, and the reward is 1 for reaching the goal and 0 otherwise (even for falling in a hole). In this example, we use the `FrozenLake-v1` environment from the `Gymnasium` library to represent the problem, and use *Q-learning* to solve it.

- Documentation for the Frozen Lake environment: https://gymnasium.farama.org/environments/toy_text/frozen_lake/

In [1]:
import numpy as np
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
import time
from tqdm.notebook import trange
from IPython import display
import matplotlib.pyplot as plt
import pickle
import ipywidgets as widgets

In [2]:
env_raw = gym.make('FrozenLake-v1', render_mode='rgb_array')  # creating the environment

In [3]:
# initializing q-table
action_space_size = env_raw.action_space.n
observation_space_size = env_raw.observation_space.n

q_table = np.zeros((observation_space_size, action_space_size))
print("Q-TABLE:")
print(q_table)

Q-TABLE:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [4]:
# hyperparameters
N_EPISODES = 10_000
MAX_STEPS_PER_EPISODE = 100

ALPHA = 0.1  # learning rate
GAMMA = 0.98  # discount rate

EPSILON = 1  # exploration rate
EPSILON_MIN = 0.001
EPSILON_DECAY = (2 * EPSILON) / N_EPISODES

LOG_RATE = N_EPISODES / 10

In [5]:
# wrap environment
rec_episodes = np.linspace(0, N_EPISODES-1, num=3, dtype=int)
trigger = lambda t: t in rec_episodes
env = RecordVideo(env_raw, video_folder="./videos", episode_trigger=trigger, disable_logger=True)

/opt/conda/lib/python3.11/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/jovyan/work/5. Temporal Difference/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/opt/conda/lib/python3.11/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.is_vector_env to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.is_vector_env` for environment variables or `env.get_attr('is_vector_env')` that will search the reminding wrappers.
  logger.warn(


## Q-learning

Q-learning combines ideas from both *Dynamic Programming* and *Monte Carlo* methods. Similarly to MC, Q-learning simulates episodes, and updates the
value function according to the returns. However, there is an important difference in the update rule of these two methods: while MC uses only returns
from the currently simulated episode, Q-learning utilizes *bootstrapping*, that is, it updates estimates based on other learned estimates, without
waiting for a final outcome.

The update rule for Q-learning looks like this:

$$ Q_t(S_t,A_t) \leftarrow Q_t(S_t,A_t) + \alpha[R_{t+1} + \gamma \max_a Q(S_{t+1}, a) - Q_t(S_t,A_t)] $$

Where $\alpha \in (0;1]$ is a constant step-size parameter and $\gamma \in [0;1]$ is the discount rate.

In [6]:
env = RecordVideo(env, video_folder="./videos", episode_trigger=trigger, disable_logger=True)
sum_rewards = 0

for episode in trange(N_EPISODES):
    state, _ = env.reset()
    done = False

    for step in range(MAX_STEPS_PER_EPISODE):
        # epsilon-greedy action selection
        if np.random.rand() > EPSILON:
            action = np.argmax(q_table[state, :])
        else:
            action = env.action_space.sample()

        new_state, reward, done, truncated, info = env.step(action)

        # updating q-table
        q_table[state, action] = q_table[state, action] * (1 - ALPHA) + \
            ALPHA * (reward + GAMMA * np.max(q_table[new_state, :]))

        state = new_state
        sum_rewards += reward

        if done:
            break

    # updating epsilon
    EPSILON = max(EPSILON - EPSILON_DECAY, EPSILON_MIN)

    # logging the results
    if (episode + 1) % LOG_RATE == 0:
        print(f'Episode {episode + 1} : avg={sum_rewards / LOG_RATE}')
        sum_rewards = 0

# saving the q-table
with open('q_table.bin', 'wb') as f:
    pickle.dump(q_table, f)

  0%|          | 0/10000 [00:00<?, ?it/s]

error: XDG_RUNTIME_DIR not set in the environment.


Episode 1000 : avg=0.014
Episode 2000 : avg=0.029
Episode 3000 : avg=0.066
Episode 4000 : avg=0.142
Episode 5000 : avg=0.408
Episode 6000 : avg=0.748
Episode 7000 : avg=0.735
Episode 8000 : avg=0.735
Episode 9000 : avg=0.742
Episode 10000 : avg=0.748


In [7]:
# Print updated Q-table
print("Q-TABLE:")
print(q_table)

Q-TABLE:
[[0.39257432 0.33122528 0.32599546 0.32968035]
 [0.24887952 0.20304953 0.2379857  0.30379654]
 [0.26761118 0.25996793 0.25930508 0.25863313]
 [0.20031616 0.14808369 0.15106146 0.24861538]
 [0.41993849 0.25519822 0.32369503 0.24139576]
 [0.         0.         0.         0.        ]
 [0.1362352  0.16193945 0.27040652 0.04143492]
 [0.         0.         0.         0.        ]
 [0.335747   0.31066246 0.36648437 0.48029629]
 [0.34107756 0.55198254 0.42440882 0.32787922]
 [0.52273099 0.30196    0.29970006 0.20900189]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.38351113 0.50529616 0.68069581 0.40917154]
 [0.64288792 0.85725197 0.6485012  0.6514231 ]
 [0.         0.         0.         0.        ]]


In [8]:
children = [widgets.Video.from_file(f'./videos/rl-video-episode-{episode}.mp4', autoplay=False, loop=False, width=500) for episode in rec_episodes]
tab = widgets.Tab()
tab.children = children
tab.titles = tuple([f'Episode {episode+1}' for episode in rec_episodes])
tab